<a href="https://colab.research.google.com/github/jb8794/NLP_ENSAE/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **I) Data exploration and processing**

Importing necessary packages

In [0]:
import numpy as np
import pandas as pd

## I.A) Exploring data on businesses

In [0]:
df_business = pd.read_json('/content/drive/My Drive/business.json', lines=True)

In [3]:
len(df_business.index)

192609

In [4]:
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


Selecting the restaurants

In [0]:
df_restaurants = df_business[df_business['categories'].str.contains('Restaurants',case=False, na=False)]

In [6]:
len(df_restaurants.index)

59371

Duplicating lines for each word in "categories" ("exploding" categories), since one restaurant may belong to several categories

In [0]:
df_restaurants_exploded = df_restaurants.assign(categories = df_restaurants.categories.str.split(', ')).explode('categories')

In [8]:
df_restaurants_exploded.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...",Specialty Food,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...",Restaurants,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...",Dim Sum,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...",Imported Food,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...",Food,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."


Exploring categories related to restaurants. In the rest of the project, we will keep the four biggest categories (in terms of "national types") : American (traditional), Italian, Mexican and Chinese

In [9]:
print(df_restaurants_exploded.categories.value_counts().to_string())

Restaurants                         59371
Food                                14800
Nightlife                            8562
Bars                                 8182
Sandwiches                           7332
Fast Food                            7257
American (Traditional)               7107
Pizza                                6804
Burgers                              5404
Breakfast & Brunch                   5381
American (New)                       4882
Italian                              4716
Mexican                              4618
Chinese                              4428
Coffee & Tea                         3647
Cafes                                3232
Japanese                             2716
Chicken Wings                        2705
Event Planning & Services            2685
Salad                                2531
Seafood                              2508
Sushi Bars                           2258
Specialty Food                       2091
Delis                             

Keeping only the categories we selected in *df_restaurants*

In [0]:
df_restaurants=df_restaurants_exploded[(df_restaurants_exploded['categories']=='American (Traditional)') | (df_restaurants_exploded['categories']=='Italian') | (df_restaurants_exploded['categories']=='Mexican') | (df_restaurants_exploded['categories']=='Chinese')]

Exploring geographical (states) categories. In the rest of the study we will only keep the five US states where there is the highest number of restaurants in the dataset : AZ (Arizona), NV (Nevada), NC (North Carolina), OH (Ohio), PA (Pennsylvania) (no Canadian state like ON Ontario)

In [11]:
print(df_business.state.value_counts().to_string())

AZ     56686
NV     36312
ON     33412
NC     14720
OH     14697
PA     11216
QC      9219
AB      8012
WI      5154
IL      1932
SC      1162
NY        22
CA        19
TX         6
XGM        4
FL         4
CT         3
AL         3
WA         3
GA         2
AK         2
VT         2
VA         2
XWY        2
NE         2
BAS        1
XGL        1
NJ         1
BC         1
CON        1
TN         1
DOW        1
NM         1
DUR        1
UT         1
AR         1


## I.B) Opening data on users reviews

Because of the very large size of the reviews.json file, we cannot use the read_json function of pandas : it exceeds the RAM available on Google Colab. Instead, we use a function which opens the json line by line.

In [0]:
import json

def init_ds(json):
    ds= {}
    keys = json.keys()
    for k in keys:
        ds[k]= []
    return ds, keys

def read_json(file):
    dataset = {}
    keys = []
    with open(file) as file_lines:
        for count, line in enumerate(file_lines):
            data = json.loads(line.strip())
            if count ==0:
                dataset, keys = init_ds(data)
            for k in keys:
                dataset[k].append(data[k])
                
        return pd.DataFrame(dataset)

In [0]:
df_reviews = read_json('/content/drive/My Drive/review.json')

In [14]:
df_reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38


In [15]:
len(df_reviews.index)

6685900

We merge the reviews with the restaurant data we opened in I.A). We merge on business_id.

In [0]:
df_reviews=pd.merge(left=df_reviews,right=df_restaurants, how='left', left_on='business_id', right_on='business_id')

We drop all the reviews lines for which we have NaNs : this corresponds to the restaurants types we did not select in I.A).

In [0]:
df_reviews=df_reviews.dropna()

In [18]:
df_reviews.head()

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,address,city,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
5,fdiNeiN_hoCxCMy2wTRW9g,w31MKYsNFMrjhWxxAb5wIw,eU_713ec6fTGNO4BegRaww,4.0,0,0,0,I'll be the first to admit that I was not exci...,2013-01-20 13:25:59,La Tavola Italiana,1 Boggs Ave,Pittsburgh,PA,15211,40.426779,-80.005652,3.5,132.0,1.0,"{'Ambience': '{'touristy': False, 'hipster': F...",Italian,"{'Tuesday': '16:0-22:0', 'Wednesday': '16:0-22..."
6,G7XHMxG0bx9oBJNECG4IFg,jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,3.0,5,4,5,Tracy dessert had a big name in Hong Kong and ...,2016-05-07 01:21:02,Tracy Dessert,"3235 Highway 7 E, Unit 29",Markham,ON,L3R 3P9,43.848936,-79.347801,3.5,122.0,1.0,"{'RestaurantsGoodForGroups': 'True', 'Restaura...",Chinese,"{'Monday': '14:0-0:0', 'Tuesday': '14:0-0:0', ..."
12,Z7wgXp98wYB57QdRY3HQ3w,GYNnVehQeXjty0xH7-6Fhw,FxLfqxdYPA6Z85PFKaqLrg,4.0,0,0,0,Wow. So surprised at the one and two star revi...,2016-05-07 01:36:53,Pazzo,1614 Washington Rd,Pittsburgh,PA,15241,40.352618,-80.051070,2.0,32.0,0.0,"{'WiFi': 'u'free'', 'Alcohol': 'u'full_bar'', ...",Italian,"{'Tuesday': '16:0-21:0', 'Wednesday': '16:0-21..."
15,svK3nBU7Rk8VfGorlrN52A,NJlxGtouq06hhC7sS2ECYw,YvrylyuWgbP90RgMqZQVnQ,5.0,0,0,0,You can't really find anything wrong with this...,2017-04-07 21:27:49,Jz's Pizza,232 Wellington Street W,Toronto,ON,M5V 3W1,43.645277,-79.389704,4.0,71.0,1.0,"{'NoiseLevel': 'u'quiet'', 'GoodForKids': 'Tru...",Italian,"{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
17,6BnQwlxRn7ZuWdzninM9sQ,JSrP-dUmLlwZiI7Dp3PQ2A,cHdJXLlKNWixBXpDwEGb_A,3.0,1,7,1,I love chinese food and I love mexican food. W...,2015-04-01 16:30:00,Chino Bandido,"15414 N 19th Ave, Ste K",Phoenix,AZ,85023,33.626831,-112.100300,4.0,1600.0,1.0,"{'RestaurantsDelivery': 'False', 'RestaurantsT...",Mexican,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."


## I.C) Opening data on revenue (from fiscal administration)

In [0]:
household_revenue=pd.read_excel('/content/drive/My Drive/USPostocodeRevenue.xlsx', header=1)

In [20]:
household_revenue.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,ARIZONA,Total,2977320.0,1368890.0,1090040.0,478170.0
2,ARIZONA,"$1 under $25,000",1038460.0,681090.0,141210.0,205240.0
3,ARIZONA,"$25,000 under $50,000",775840.0,388600.0,193940.0,179410.0
4,ARIZONA,"$50,000 under $75,000",410210.0,162320.0,186090.0,54590.0


Rearranging the table extracted from excel : we get a table where each rows corresponds to a postcode area, each column corresponds to the number of households in a given revenue bracket : from 0 to 25000 dollars , 25000 to 50000 dollars ...

In [0]:
postcode_revenue = pd.DataFrame(columns=['ZipCode','number_taxpayers_postcode', '1_to_25000', '25000_to_50000','50000_to_75000','75000_to_100000','100000_to_200000','200000_and_more'])
for i in range(int((len(household_revenue.index)-1)/8)):
  postcode_revenue=postcode_revenue.append({'ZipCode': household_revenue.iloc[(1+8*i),0],'number_taxpayers_postcode': household_revenue.iloc[(1+8*i),2], '1_to_25000': household_revenue.iloc[(2+8*i),2], '25000_to_50000': household_revenue.iloc[(3+8*i),2], '50000_to_75000': household_revenue.iloc[(4+8*i),2],'75000_to_100000': household_revenue.iloc[(5+8*i),2],'100000_to_200000': household_revenue.iloc[(6+8*i),2],'200000_and_more':household_revenue.iloc[(7+8*i),2] }, ignore_index=True)

Converting the table values to type *float* to compute new features.

In [0]:
postcode_revenue['number_taxpayers_postcode']=postcode_revenue['number_taxpayers_postcode'].astype(float)
postcode_revenue['1_to_25000']=postcode_revenue['1_to_25000'].astype(float)
postcode_revenue['25000_to_50000']=postcode_revenue['25000_to_50000'].astype(float)
postcode_revenue['50000_to_75000']=postcode_revenue['50000_to_75000'].astype(float)
postcode_revenue['75000_to_100000']=postcode_revenue['75000_to_100000'].astype(float)
postcode_revenue['100000_to_200000']=postcode_revenue['100000_to_200000'].astype(float)
postcode_revenue['200000_and_more']=postcode_revenue['200000_and_more'].astype(float)

We compute the three following features:

*   postcode_average_revenue : average revenue per postcode area (taking for each bracket the center of the bracket, e.g. 12500 for [0,25000])
*   proportion_low_revenue : proportion of households earning less than 25000 dollar per year in a postcode area
*   proportion_high_revenue : proportion of households earning more than 100000 dollar per year in a postcode area


In [0]:
postcode_revenue['postcode_average_revenue']=(postcode_revenue['1_to_25000']*12500+postcode_revenue['25000_to_50000']*37500+postcode_revenue['50000_to_75000']*62500+postcode_revenue['75000_to_100000']*87500+postcode_revenue['100000_to_200000']*150000+postcode_revenue['200000_and_more']*400000)/(postcode_revenue['number_taxpayers_postcode'])
postcode_revenue['proportion_low_revenue']=100*postcode_revenue['1_to_25000']/postcode_revenue['number_taxpayers_postcode']
postcode_revenue['proportion_high_revenue']=100*(postcode_revenue['100000_to_200000']+postcode_revenue['200000_and_more'])/postcode_revenue['number_taxpayers_postcode']

In [24]:
postcode_revenue.head()

,ZipCode,number_taxpayers_postcode,1_to_25000,25000_to_50000,50000_to_75000,75000_to_100000,100000_to_200000,200000_and_more,postcode_average_revenue,proportion_low_revenue,proportion_high_revenue
0,ARIZONA,2977320.0,1038460.0,775840.0,410210.0,260590.0,367950.0,124270.0,65634.530383,34.879019,16.532318
1,85003,4030.0,1250.0,990.0,540.0,340.0,580.0,330.0,83188.585608,31.017370,22.580645
2,85004,3420.0,1030.0,900.0,520.0,300.0,450.0,220.0,76279.239766,30.116959,19.590643
3,85006,9660.0,4410.0,3070.0,1170.0,470.0,460.0,80.0,39906.832298,45.652174,5.590062
4,85007,5440.0,2510.0,1510.0,510.0,280.0,400.0,230.0,54480.698529,46.139706,11.580882


In [0]:
postcode_revenue=postcode_revenue.drop(['1_to_25000','25000_to_50000','50000_to_75000','75000_to_100000','100000_to_200000','200000_and_more'],axis=1)

In [26]:
postcode_revenue.tail()

,ZipCode,number_taxpayers_postcode,postcode_average_revenue,proportion_low_revenue,proportion_high_revenue
3505,19607,11790.0,62171.331637,31.552163,14.503817
3506,19608,11920.0,86419.882550,29.110738,26.258389
3507,19609,5420.0,59547.970480,31.365314,13.653137
3508,19610,8110.0,101926.633785,26.633785,30.086313
3509,19611,4900.0,43061.224490,42.040816,6.122449


## I.D) Merging databases

Selecting reviews in the five US States we chose : Arizona, Nevada, North Carolina, Ohio, Pennsylvania

In [0]:
df_reviews['state']=df_reviews['state'].astype(str)
df_reviews=df_reviews[(df_reviews['state']=='AZ') | (df_reviews['state']=='NV') | (df_reviews['state']=='NC') | (df_reviews['state']=='OH') | (df_reviews['state']=='PA')]

Getting rid of lines with weird postcode values in the reviews database : a restaurant located in Ohio has a canadian type postcode! (M8Z 1N1)

In [0]:
df_reviews=df_reviews[df_reviews['postal_code']!='M8Z 1N1']
df_reviews=df_reviews[df_reviews['postal_code']!='']

Getting rid of some rows in the postcode database (they correspond to whole states instead of postcode areas)

In [0]:
for u in ['ARIZONA','NEVADA','NORTH CAROLINA','OHIO','PENNSYLVANIA']:
  postcode_revenue=postcode_revenue[postcode_revenue['ZipCode']!=u]

Converting postcode columns from *df_reviews* and *postcode_revenue* to same type to prepare for merge.

In [0]:
df_reviews['postal_code']=df_reviews['postal_code'].astype(int)
postcode_revenue['ZipCode']=postcode_revenue['ZipCode'].astype(int)

Merging reviews with postcode review data

In [0]:
df=pd.merge(left=df_reviews,right=postcode_revenue, how='left', left_on='postal_code', right_on='ZipCode')

In [39]:
df.head()

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,address,city,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours,ZipCode,number_taxpayers_postcode,postcode_average_revenue,proportion_low_revenue,proportion_high_revenue
0,fdiNeiN_hoCxCMy2wTRW9g,w31MKYsNFMrjhWxxAb5wIw,eU_713ec6fTGNO4BegRaww,4.0,0,0,0,I'll be the first to admit that I was not exci...,2013-01-20 13:25:59,La Tavola Italiana,1 Boggs Ave,Pittsburgh,PA,15211,40.426779,-80.005652,3.5,132.0,1.0,"{'Ambience': '{'touristy': False, 'hipster': F...",Italian,"{'Tuesday': '16:0-22:0', 'Wednesday': '16:0-22...",15211.0,6340.0,61632.492114,30.126183,12.933754
1,Z7wgXp98wYB57QdRY3HQ3w,GYNnVehQeXjty0xH7-6Fhw,FxLfqxdYPA6Z85PFKaqLrg,4.0,0,0,0,Wow. So surprised at the one and two star revi...,2016-05-07 01:36:53,Pazzo,1614 Washington Rd,Pittsburgh,PA,15241,40.352618,-80.051070,2.0,32.0,0.0,"{'WiFi': 'u'free'', 'Alcohol': 'u'full_bar'', ...",Italian,"{'Tuesday': '16:0-21:0', 'Wednesday': '16:0-21...",15241.0,10580.0,137370.037807,25.425331,44.328922
2,6BnQwlxRn7ZuWdzninM9sQ,JSrP-dUmLlwZiI7Dp3PQ2A,cHdJXLlKNWixBXpDwEGb_A,3.0,1,7,1,I love chinese food and I love mexican food. W...,2015-04-01 16:30:00,Chino Bandido,"15414 N 19th Ave, Ste K",Phoenix,AZ,85023,33.626831,-112.100300,4.0,1600.0,1.0,"{'RestaurantsDelivery': 'False', 'RestaurantsT...",Mexican,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",85023.0,15510.0,60694.713088,34.429400,14.313346
3,6BnQwlxRn7ZuWdzninM9sQ,JSrP-dUmLlwZiI7Dp3PQ2A,cHdJXLlKNWixBXpDwEGb_A,3.0,1,7,1,I love chinese food and I love mexican food. W...,2015-04-01 16:30:00,Chino Bandido,"15414 N 19th Ave, Ste K",Phoenix,AZ,85023,33.626831,-112.100300,4.0,1600.0,1.0,"{'RestaurantsDelivery': 'False', 'RestaurantsT...",Chinese,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",85023.0,15510.0,60694.713088,34.429400,14.313346
4,Amo5gZBvCuPc_tZNpHwtsA,DzZ7piLBF-WsJxqosfJgtA,qx6WhZ42eDKmBchZDax4dQ,5.0,1,0,0,"Our family LOVES the food here. Quick, friendl...",2017-03-27 01:14:37,Casanova Brothers Pizza,959 N Val Vista Dr,Gilbert,AZ,85234,33.367708,-111.755830,4.5,943.0,1.0,"{'RestaurantsPriceRange2': '1', 'OutdoorSeatin...",Italian,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",85234.0,22250.0,83640.449438,28.359551,25.752809


In [33]:
len(df.index)

1443571

We drop reviews appearing on more than one row : this means that they correspond to restaurants belonging to different categories at the same time (for exemple, restaurants serving both mexican and chinese food).

In [0]:
df=df.drop_duplicates(subset=['review_id'], keep=False)

In [42]:
len(df.index)

1318654

Dropping columns we won't use

In [0]:
df=df.drop(['latitude','longitude','ZipCode'], axis=1)

In [45]:
df.head()

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,address,city,state,postal_code,stars_y,review_count,is_open,attributes,categories,hours,number_taxpayers_postcode,postcode_average_revenue,proportion_low_revenue,proportion_high_revenue
0,fdiNeiN_hoCxCMy2wTRW9g,w31MKYsNFMrjhWxxAb5wIw,eU_713ec6fTGNO4BegRaww,4.0,0,0,0,I'll be the first to admit that I was not exci...,2013-01-20 13:25:59,La Tavola Italiana,1 Boggs Ave,Pittsburgh,PA,15211,3.5,132.0,1.0,"{'Ambience': '{'touristy': False, 'hipster': F...",Italian,"{'Tuesday': '16:0-22:0', 'Wednesday': '16:0-22...",6340.0,61632.492114,30.126183,12.933754
1,Z7wgXp98wYB57QdRY3HQ3w,GYNnVehQeXjty0xH7-6Fhw,FxLfqxdYPA6Z85PFKaqLrg,4.0,0,0,0,Wow. So surprised at the one and two star revi...,2016-05-07 01:36:53,Pazzo,1614 Washington Rd,Pittsburgh,PA,15241,2.0,32.0,0.0,"{'WiFi': 'u'free'', 'Alcohol': 'u'full_bar'', ...",Italian,"{'Tuesday': '16:0-21:0', 'Wednesday': '16:0-21...",10580.0,137370.037807,25.425331,44.328922
4,Amo5gZBvCuPc_tZNpHwtsA,DzZ7piLBF-WsJxqosfJgtA,qx6WhZ42eDKmBchZDax4dQ,5.0,1,0,0,"Our family LOVES the food here. Quick, friendl...",2017-03-27 01:14:37,Casanova Brothers Pizza,959 N Val Vista Dr,Gilbert,AZ,85234,4.5,943.0,1.0,"{'RestaurantsPriceRange2': '1', 'OutdoorSeatin...",Italian,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",22250.0,83640.449438,28.359551,25.752809
5,S337tATeouQJdoPYyir29w,2mxBNBeFrgDszqGS5tdEHA,d_L-rfS1vT3JMzgCUGtiow,5.0,0,0,0,Pick any meat on the planet and the chef will ...,2016-07-25 03:57:19,Border Grill,3950 S Las Vegas Blvd,Las Vegas,NV,89119,4.0,1623.0,1.0,"{'WiFi': 'u'free'', 'RestaurantsTakeOut': 'Tru...",Mexican,"{'Monday': '10:0-22:0', 'Tuesday': '11:0-22:0'...",21470.0,35735.910573,49.464369,3.772706
6,qm97yMwREr7BKkexlwLFbg,TTRVdTXKcq-xn6-1IWbwEw,Gyrez6K8f1AyR7dzW9fvAw,5.0,0,0,1,their pettuccine was fresh-made in the morning...,2013-12-28 22:28:08,Casanova,"3377 S Las Vegas Blvd, Ste 2170",Las Vegas,NV,89109,4.0,495.0,1.0,"{'HasTV': 'True', 'Ambience': '{'romantic': Fa...",Italian,"{'Monday': '8:0-23:0', 'Tuesday': '8:0-23:0', ...",3460.0,95881.502890,37.283237,24.277457


Saving our database (pickle format more convenient)

In [0]:
save_url = "/content/drive/My Drive/yelp_data_29032020.pkl"
df.to_pickle(save_url)

**II) Prediction of business characteristics based on user's reviews**

**III) Tip generation**